In [88]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

pd.set_option('display.max_columns', 500)

# Specify data locations

In [89]:
#Specify the path to clinical_data_cleaned.csv, which contains clinical features and outcome data
clinical_data_path = "/deep/group/aihc-bootcamp-winter2020/dlbcl/DLBCL-Morph/clinical_data_cleaned.csv"

#Specify the path to cell_shapes.csv, which contains geometric features for each tumor nucleus
cell_shapes_path = "/deep/group/aihc-bootcamp-winter2020/dlbcl/DLBCL-Morph/Cells/cell_shapes.csv"

# Select which features to use

In [90]:
#Specify one of the following: "ALL", "GEOMETRIC", "CLINICAL"
FEATURES = "GEOMETRIC"

# Load data

In [91]:
df_outcome = pd.read_csv(clinical_data_path)
print(df_outcome.columns)

Index(['patient_id', 'MYC IHC', 'BCL2 IHC', 'BCL6 IHC', 'CD10 IHC', 'MUM1 IHC',
       'HANS', 'BCL6 FISH', 'MYC FISH', 'BCL2 FISH', 'Age', 'ECOG PS', 'LDH',
       'EN', 'Stage', 'IPI Score', 'IPI Risk Group (4 Class)',
       'RIPI Risk Group', 'OS', 'PFS', 'Follow-up Status'],
      dtype='object')


In [92]:
df_outcome = df_outcome.fillna(df_outcome.mean()) #fill NaN entries with the mean value of the feature

In [93]:
df_cell_shapes = pd.read_csv(cell_shapes_path)
df_cell_shapes.columns

Index(['patient_id', 'patch_id', 'cell_number', 'rotate_angle', 'shortAxis',
       'longAxis', 'ellip_perimt', 'ellip_area', 'hull_area', 'minDiameter',
       'maxDiameter', 'minAngle', 'maxAngle', 'esf', 'csf', 'sf1', 'sf2',
       'elogation', 'convexity', 'rectCenter_x', 'rectCenter_y', 'rect_width',
       'rect_height', 'ellip_centroid_x', 'ellip_centroid_y'],
      dtype='object')

In [94]:
print("Number of patients before join:", df_cell_shapes['patient_id'].nunique())
new_cols = ['MYC IHC', 'BCL2 IHC', 'BCL6 IHC', 'CD10 IHC', 'MUM1 IHC',
       'HANS', 'BCL6 FISH', 'MYC FISH', 'BCL2 FISH', 'Age', 'ECOG PS', 'LDH',
       'EN', 'Stage', 'IPI Score', 'IPI Risk Group (4 Class)',
       'RIPI Risk Group']

df = df_cell_shapes.merge(df_outcome[['patient_id', 'OS', 'Follow-up Status'] + new_cols], left_on='patient_id', right_on='patient_id', how='inner', validate="m:1")
print("Number of patients after join:", df['patient_id'].nunique())

df = df.drop(["patch_id", "cell_number"], axis=1)
df = df.astype(np.float)

df_agg = df.groupby("patient_id").agg([np.mean, np.std])
display(df_agg.head(20))
df_agg.columns = df_agg.columns.map('_'.join)
df_agg = df_agg.drop(["OS_std", "Follow-up Status_std"] + [col + '_std' for col in new_cols], axis=1)
df_agg = df_agg.rename(columns={"OS_mean":"OS", "Follow-up Status_mean": "FUS"})
display(df_agg.head(20))

Number of patients before join: 170
Number of patients after join: 170


rotate_angle             shortAxis             longAxis             \
                   mean        std       mean       std       mean        std   
patient_id                                                                      
13901.0      -30.203768  32.123104  18.053119  5.319929  25.791760   7.707645   
13902.0      -29.423052  31.866907  16.799208  4.851827  24.317176   7.529572   
13903.0      -30.349564  31.280504  15.809540  4.716802  24.220746   7.749293   
13904.0      -34.466977  34.333292  15.451094  5.098248  25.875319   8.977452   
13905.0      -26.744428  30.938156  16.226045  4.673878  26.082492   8.333628   
13906.0      -31.406059  31.784805  14.054645  4.444628  23.142015   7.776459   
13908.0      -28.916903  31.056615  16.208309  4.012041  23.276012   6.442699   
13911.0      -32.826387  32.435577  19.449140  7.167019  29.794954  10.976688   
13912.0      -28.596861  31.305722  16.743245  5.374643  24.101754   7.804513   
13913.0      -29.798667  32.295170  19.521436  6.621578  27.932794   9.193201   
13914.0      -28.222480  31.463109  18.275726  5.239078  26.187776   7.476985   
13915.0      -29.611584  32.557601  17.298002  5.574451  26.275516   8.599822   
13917.0      -27.896866  30.499674  17.724260  4.807410  25.605818   7.736457   
13919.0      -29.634783  31.941830  20.516365  5.790033  28.444294   7.450097   
13920.0      -32.828376  32.673702  19.059800  6.573795  28.220732  10.043003   
13922.0      -32.258712  32.496303  14.490485  4.649048  21.873130   7.013926   
13923.0      -27.832473  31.534587  16.767613  5.211577  23.858362   7.794775   
13924.0      -30.639950  32.441900  18.679329  6.553707  27.530820   9.683460   
13926.0      -30.545457  31.476047  14.865614  5.250688  23.246220   8.018779   
13927.0      -28.623466  31.955828  15.333754  4.561883  24.467952   7.762447   

           ellip_perimt             ellip_area               hull_area  \
                   mean        std        mean         std        mean   
patient_id                                                               
13901.0       69.604697  19.242608  388.327122  207.450218  367.746142   
13902.0       65.307907  18.409229  341.227716  180.928765  326.058674   
13903.0       63.808208  18.283335  318.852182  172.578439  303.356232   
13904.0       66.201771  20.961549  337.499648  196.429641  310.527981   
13905.0       67.598635  19.232158  351.702822  196.199166  331.095023   
13906.0       59.534827  18.056534  272.489293  164.586702  265.656800   
13908.0       62.689855  15.285239  309.134267  147.318309  298.875458   
13911.0       78.496813  27.154253  500.461950  325.232429  466.986029   
13912.0       64.892541  19.485145  340.142098  194.188423  325.342362   
13913.0       75.326885  23.759126  465.690314  288.777426  439.633221   
13914.0       70.594268  18.672403  396.950162  195.795426  378.092128   
13915.0       69.396256  21.071611  383.612964  221.934273  362.943152   
13917.0       68.819582  18.566574  376.252781  198.748972  357.228147   
13919.0       77.613950  19.337635  481.918010  226.966118  457.540421   
13920.0       75.179342  25.104380  462.729583  296.848608  446.300729   
13922.0       57.907799  17.198423  266.360954  154.019881  256.424783   
13923.0       64.485298  19.423247  337.717670  211.015033  322.422059   
13924.0       73.473401  24.428615  442.278915  278.418533  420.794104   
13926.0       60.810599  19.733711  294.834549  188.824507  289.056596   
13927.0       63.567290  18.181448  312.459563  176.431116  300.399705   

                       minDiameter           maxDiameter             \
                   std        mean       std        mean        std   
patient_id                                                            
13901.0     209.657477   18.507646  5.741539   27.229503   7.620760   
13902.0     188.067495   17.360127  5.360700   25.801860   7.542590   
13903.0     178.119444   16.431440  5.011610   25.691504   7.870433   
13904.0     187.39

,rotate_angle_mean,rotate_angle_std,shortAxis_mean,shortAxis_std,longAxis_mean,longAxis_std,ellip_perimt_mean,ellip_perimt_std,ellip_area_mean,ellip_area_std,hull_area_mean,hull_area_std,minDiameter_mean,minDiameter_std,maxDiameter_mean,maxDiameter_std,minAngle_mean,minAngle_std,maxAngle_mean,maxAngle_std,esf_mean,esf_std,csf_mean,csf_std,sf1_mean,sf1_std,sf2_mean,sf2_std,elogation_mean,elogation_std,convexity_mean,convexity_std,rectCenter_x_mean,rectCenter_x_std,rectCenter_y_mean,rectCenter_y_std,rect_width_mean,rect_width_std,rect_height_mean,rect_height_std,ellip_centroid_x_mean,ellip_centroid_x_std,ellip_centroid_y_mean,ellip_centroid_y_std,OS,FUS,MYC IHC_mean,BCL2 IHC_mean,BCL6 IHC_mean,CD10 IHC_mean,MUM1 IHC_mean,HANS_mean,BCL6 FISH_mean,MYC FISH_mean,BCL2 FISH_mean,Age_mean,ECOG PS_mean,LDH_mean,EN_mean,Stage_mean,IPI Score_mean,IPI Risk Group (4 Class)_mean,RIPI Risk Group_mean
patient_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13901.0,-30.203768,32.123104,18.053119,5.319929,25.791760,7.707645,69.604697,19.242608,388.327122,207.450218,367.746142,209.657477,18.507646,5.741539,27.229503,7.620760,-23.598508,76.783746,-1.167264,140.147801,0.714059,0.144262,0.939780,0.068678,0.670788,0.129709,0.685801,0.141318,1.540486,0.422075,1.046642,0.127535,10.966762,3.843282,10.444223,3.633004,21.168709,7.746182,21.041199,7.518230,10.971682,4.132051,10.463238,3.902402,2.78,0.0,0.000000,30.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,61.0,2.0,1.0,1.0,4.0,4.0,3.0,2.0
13902.0,-29.423052,31.866907,16.799208,4.851827,24.317176,7.529572,65.307907,18.409229,341.227716,180.928765,326.058674,188.067495,17.360127,5.360700,25.801860,7.542590,-21.145773,76.969812,14.030860,136.402492,0.708758,0.143803,0.938162,0.066646,0.662463,0.128765,0.681033,0.138314,1.547771,0.416758,1.043724,0.119947,10.075676,3.519298,10.073096,3.726110,19.927231,7.463325,19.796531,7.413028,10.078738,3.748831,10.066373,3.953555,10.08,0.0,10.000000,0.000000,70.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.224359,24.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
13903.0,-30.349564,31.280504,15.809540,4.716802,24.220746,7.749293,63.808208,18.283335,318.852182,172.578439,303.356232,178.119444,16.431440,5.011610,25.691504,7.870433,-13.327497,78.365459,-19.692376,136.967791,0.676833,0.159982,0.920523,0.080190,0.633071,0.143944,0.653426,0.134246,1.609098,0.406415,1.043825,0.119860,9.909315,3.679379,9.807300,3.662121,18.741856,7.104246,19.873945,8.253596,9.912469,4.006994,9.762954,3.802286,9.78,0.0,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.213483,0.078212,0.000000,25.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
13904.0,-34.466977,34.333292,15.451094,5.098248,25.875319,8.977452,66.201771,20.961549,337.499648,196.429641,310.527981,187.398678,15.933636,5.182056,27.229388,8.842433,-29.878338,79.497370,8.483001,145.738244,0.620706,0.151584,0.894926,0.091007,0.584008,0.141612,0.603398,0.141624,1.778404,0.566419,1.057293,0.141543,11.102816,4.587523,9.102394,3.756669,20.360341,8.722106,19.286392,8.926088,10.985653,4.938259,8.977256,4.204336,7.40,1.0,14.461538,48.825641,23.455497,0.380435,0.551351,0.432432,0.213483,0.078212,0.224359,64.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
13905.0,-26.744428,30.938156,16.226045,4.673878,26.082492,8.333628,67.598635,19.232158,351.702822,196.199166,331.095023,197.254295,16.972882,4.714176,27.169452,8.351487,-22.836169,80.285581,26.910928,145.588529,0.645798,0.153842,0.907719,0.083815,0.615255,0.138939,0.642958,0.135344,1.640414,0.431812,1.044817,0.116173,11.424346,4.451223,9.273681,3.150679,21.901859,8.921579,18.706747,6.952829,11.507091,4.821830,9.255364,3.503049,9.62,0.0,14.461538,48.825641,23.455497,0.380435,0.551351,0.432432,0.000000,0.000000,0.000000,41.0,1.0,0.0,1.0,4.0,1.0,0.0,1.0
13906.0,-31.406059,31.784805,14.054645,4.444628,23.142015,7.776459,59.534827,18.056534,272.489293,164.586702,265.656800,171.538904,15.060175,4.739558,24.769079,8.013515,-22.669314,80.650718,21.388682,143.355495,0.631809,0.156515,0.899429,0.0899

# C-index and optimism computation

In [95]:
geo_cols = ['rotate_angle_mean', 'rotate_angle_std', 'shortAxis_mean',
       'shortAxis_std', 'longAxis_mean', 'longAxis_std', 'ellip_perimt_mean',
       'ellip_perimt_std', 'ellip_area_mean', 'ellip_area_std',
       'hull_area_mean', 'hull_area_std', 'minDiameter_mean',
       'minDiameter_std', 'maxDiameter_mean', 'maxDiameter_std',
       'minAngle_mean', 'minAngle_std', 'maxAngle_mean', 'maxAngle_std',
       'esf_mean', 'esf_std', 'csf_mean', 'csf_std', 'sf1_mean', 'sf1_std',
       'sf2_mean', 'sf2_std', 'elogation_mean', 'elogation_std',
       'convexity_mean', 'convexity_std', 'rectCenter_x_mean',
       'rectCenter_x_std', 'rectCenter_y_mean', 'rectCenter_y_std',
       'rect_width_mean', 'rect_width_std', 'rect_height_mean',
       'rect_height_std', 'ellip_centroid_x_mean', 'ellip_centroid_x_std',
       'ellip_centroid_y_mean', 'ellip_centroid_y_std', 'OS', 'FUS']

In [96]:
penalizer_g_q = 0 # Geomatrical features
penalizer_c_q = 0  # Clinical features
penalizer_gc_q = 0.00005  # All features

In [97]:
if FEATURES == "ALL":
    penalizer = penalizer_gc_q
    df = df_agg
elif FEATURES == "GEOMETRIC":
    penalizer = penalizer_g_q
    df = df_agg[geo_cols]
else:
    penalizer = penalizer_c_q
    df = df_agg.drop(set(geo_cols) - set(["OS", "FUS"]), axis=1)

In [98]:
from sklearn.preprocessing import QuantileTransformer

In [99]:
qt = QuantileTransformer(n_quantiles=10, random_state=42)
qt.fit(df)

df = pd.DataFrame.from_records(qt.transform(df), columns=df.columns)

In [100]:
df.columns

Index(['rotate_angle_mean', 'rotate_angle_std', 'shortAxis_mean',
       'shortAxis_std', 'longAxis_mean', 'longAxis_std', 'ellip_perimt_mean',
       'ellip_perimt_std', 'ellip_area_mean', 'ellip_area_std',
       'hull_area_mean', 'hull_area_std', 'minDiameter_mean',
       'minDiameter_std', 'maxDiameter_mean', 'maxDiameter_std',
       'minAngle_mean', 'minAngle_std', 'maxAngle_mean', 'maxAngle_std',
       'esf_mean', 'esf_std', 'csf_mean', 'csf_std', 'sf1_mean', 'sf1_std',
       'sf2_mean', 'sf2_std', 'elogation_mean', 'elogation_std',
       'convexity_mean', 'convexity_std', 'rectCenter_x_mean',
       'rectCenter_x_std', 'rectCenter_y_mean', 'rectCenter_y_std',
       'rect_width_mean', 'rect_width_std', 'rect_height_mean',
       'rect_height_std', 'ellip_centroid_x_mean', 'ellip_centroid_x_std',
       'ellip_centroid_y_mean', 'ellip_centroid_y_std', 'OS', 'FUS'],
      dtype='object')

## Un-corrected C-index

In [101]:
cph = CoxPHFitter(penalizer=penalizer)
cph.fit(df, duration_col='OS', event_col='FUS')
c_main = cph.score(df, scoring_method="concordance_index")
print("The un-corrected C-index is: %.3f" % c_main)

The un-corrected C-index is: 0.786


/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)


## Optimism computation

In [102]:
np.random.seed(42)
c_b_boot, c_b_orig = [], []
num_bootstraps = 1000
bootstrap_size = len(df)

for i in range(num_bootstraps):
    choices = np.random.choice(np.arange(0, len(df)), size=bootstrap_size, replace=True) 
    new_df = df.iloc[choices] #sample bootstrap replicate with replacement
    
    cph = CoxPHFitter(penalizer=penalizer)
    cph.fit(new_df, duration_col='OS', event_col='FUS') #fit on bootstrap
    
    c = cph.score(new_df, scoring_method="concordance_index") #score on bootstrap
    c_b_boot.append(c)
    
    c = cph.score(df, scoring_method="concordance_index") #score on original
    c_b_orig.append(c)
    
c_b_orig = np.array(c_b_orig)
c_b_boot = np.array(c_b_boot)

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWa

...

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWa

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__ini

In [103]:
o = np.mean(c_b_boot - c_b_orig)
print("measure of optimism: %.3f" % o)

measure of optimism: 0.152


In [104]:
c_final = c_main - o
print("optimism-corrected c-index is %.3f" % c_final)

optimism-corrected c-index is 0.635


# 95% Confidence Intervals

In [105]:
np.random.seed(42)
c_indices = []
num_bootstraps = 1000
bootstrap_size = len(df)

cph = CoxPHFitter(penalizer=penalizer) #fit on original data
cph.fit(df, duration_col='OS', event_col='FUS')

for i in range(num_bootstraps):
    choices = np.random.choice(np.arange(0, len(df)), size=bootstrap_size, replace=True)
    new_df = df.iloc[choices]
    
    c_index = cph.score(new_df, scoring_method="concordance_index")
    c_indices.append(c_index)
    
c_indices.sort()
hi = c_indices[974]
lo = c_indices[24]

print("Confidence interval is: %.3f - %.3f" % (lo, hi))

/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/fitters/coxph_fitter.py:976: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently in 500 steps.

  warnings.warn("Newton-Rhaphson failed to converge sufficiently in %d steps.\n" % max_steps, utils.ConvergenceWarning)
/deep/group/packages/miniconda3/envs/dlbcl/lib/python3.7/site-packages/lifelines/utils/__init__.py:940: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")


Confidence interval is: 0.726 - 0.843


In [106]:
print("Adjusting for optimism")
print("Adjusted confidence interval is: %.3f - %.3f" % (lo-o, hi-o))

Adjusting for optimism
Adjusted confidence interval is: 0.574 - 0.691


# Results

# Clinical features only
Optimism = 0.074

Final c-index: 0.674

Adjusted 95% CI: (0.602, 0.737)

# Geometric features only
Optimism = 0.152

Final c-index = 0.635

Adjusted 95% CI: (0.574, 0.691)

# Clinical + Geometric features
Optimism = 0.156

Final c-index = 0.700

Adjusted 95% CI: (0.651, 0.744)